In [33]:
# !pip install xarray==0.19.0 
# !pip install climetlab
# !pip install climetlab_weatherbench
# !pip install numexpr=='2.7.3'

In [34]:
import xarray as xr

In [35]:
def temperature_unnormalize(data):
    true_temp = xr.open_mfdataset('data/us_temperature_forecast_true.nc')
    m, s = true_temp.mean().load(), true_temp.std().load()
    data = data.assign_coords(lon=(((data.lon + 180) % 360) - 180))
    data = (data - data.mean().load())/data.std().load()
    data = data * s + m - 273.15
    data.rename({"t2m" : "celcius"})
    return data

def precipitation_unnormalize(data):
    true_temp = xr.open_mfdataset('data/us_precipitation_forecast_true.nc')
    m, s = true_temp.mean().load(), true_temp.std().load()
    data = data.assign_coords(lon=(((data.lon + 180) % 360) - 180))
    data = (data - data.mean().load())/data.std().load()
    data = data * s + m
#     data.rename({"t2m" : "celcius"})
    return data

In [36]:
files = ['us_temperature_ann_forecast.nc', 
'us_temperature_cnn_forecast.nc', 
'us_temperature_ann_forecast.nc', 
'us_temperature_hourly_climatology_forecast.nc', 
'us_temperature_persistence_forecast.nc', 
'us_temperature_resnet_forecast.nc',
'us_temperature_forecast_true.nc']

In [37]:
temp_preds = []
names = ['cnn', 'persistence', 'ann', 'resnet', 'climatology', 'linreg', 'true']
for i in range(len(files)):
    files[i] = 'data/' + files[i]
    print(files[i])
    a = xr.open_mfdataset(files[i])
    a = temperature_unnormalize(a).t2m
    a.name = names[i]
    temp_preds.append(a)
pred_temp = xr.merge(temp_preds)

data/us_temperature_ann_forecast.nc
data/us_temperature_cnn_forecast.nc
data/us_temperature_ann_forecast.nc
data/us_temperature_hourly_climatology_forecast.nc
data/us_temperature_persistence_forecast.nc
data/us_temperature_resnet_forecast.nc
data/us_temperature_forecast_true.nc


In [38]:
pred_temp.to_dataframe().to_csv('forecasts/temperature_forecasts.csv')

In [39]:
for n in names[:-1]:
    error = pred_temp[n].values.flatten() - pred_temp.true.values.flatten()
    print(n, (error.T@error)**0.5)

cnn 1481.253523202561
persistence 1269.1903915488804
ann 1481.253523202561
resnet 1279.7226750354937
climatology 1662.1070212233628
linreg 1252.4536418566556


In [40]:
files = ['us_precipitation_ann_forecast.nc', 
'us_precipitation_cnn_forecast.nc', 
'us_precipitation_ann_forecast.nc', 
'us_precipitation_hourly_climatology_forecast.nc', 
'us_precipitation_persistence_forecast.nc', 
'us_precipitation_resnet_forecast.nc',
'us_precipitation_forecast_true.nc']

In [41]:
temp_preds = []
names = ['cnn', 'persistence', 'ann', 'resnet', 'climatology', 'linreg', 'true']
for i in range(len(files)):
    files[i] = 'data/' + files[i]
    print(files[i])
    a = xr.open_mfdataset(files[i])
    a = precipitation_unnormalize(a).tp
    a.name = names[i]
    temp_preds.append(a)
pred_temp = xr.merge(temp_preds)

data/us_precipitation_ann_forecast.nc
data/us_precipitation_cnn_forecast.nc
data/us_precipitation_ann_forecast.nc
data/us_precipitation_hourly_climatology_forecast.nc
data/us_precipitation_persistence_forecast.nc
data/us_precipitation_resnet_forecast.nc
data/us_precipitation_forecast_true.nc


In [42]:
pred_temp.to_dataframe().to_csv('forecasts/precipitation_forecasts.csv')

In [43]:
for n in names[:-1]:
    error = pred_temp[n].values.flatten() - pred_temp.true.values.flatten()
    print(n, (error.T@error)**0.5)

cnn 0.1453465201434373
persistence 0.14119590003856441
ann 0.1453465201434373
resnet 0.14007755851737438
climatology 0.15616280387689155
linreg 0.1424659196041572
